# SQL ПЛЭЙБУК: Полный справочник для собеседований и работы

**PostgreSQL** — стандарт для аналитики + явные различия с MySQL.

## СОДЕРЖАНИЕ
[1. Фундаментальные основы](#раздел-1-фундаментальные-основы)  
[2. Выборка и фильтрация данных](#раздел-2-выборка-и-фильтрация-данных)  
[3. Агрегация и группировка](#раздел-3-агрегация-и-группировка)  
[4. Соединения таблиц (JOINs)](#раздел-4-соединения-таблиц-joins)  
[5. Подзапросы (Subqueries)](#раздел-5-подзапросы-subqueries)  
[6. CTE (Common Table Expressions)](#раздел-6-cte-common-table-expressions)  
[7. Оконные функции](#раздел-7-оконные-функции)  
[8. Оптимизация и индексы](#раздел-8-оптимизация-и-индексы)  
[9. Транзакции и управление](#раздел-9-транзакции-и-управление)  
[10. Работа с датами и строками](#раздел-10-работа-с-датами-и-строками)  
[11. Продвинутые темы](#раздел-11-продвинутые-темы)  
[12. Практические паттерны и задачи](#раздел-12-практические-паттерны-и-задачи)  
[Приложения](#приложения)

---
**Источники:** PostgreSQL docs, MySQL docs, LeetCode SQL.  
*Добавляйте свои шаблоны через PR.*

## РАЗДЕЛ 1: ФУНДАМЕНТАЛЬНЫЕ ОСНОВЫ
### 1.1. SQL и реляционные базы данных
**Что такое SQL?**
SQL (Structured Query Language) — декларативный язык программирования для работы с реляционными базами данных. Позволяет:
- Создавать и изменять структуру БД (DDL – Data Definition Language)
- Управлять данными (DML – Data Manipulation Language)
- Контролировать доступ (DCL – Data Control Language)
- Управлять транзакциями (TCL – Transaction Control Language)

**Отличие SQL от MySQL?**
- **SQL** — стандартизированный язык запросов (ANSI/ISO стандарт)
- **MySQL** — конкретная СУБД (система управления базами данных), реализующая стандарт SQL

**Реляционная база данных** — данные организованы в таблицы (отношения), связанные ключами. Основные понятия:
- Таблица (Table) — совокупность строк и столбцов
- Строка (Row/Record) — одна запись в таблице
- Столбец (Column/Field) — атрибут записи
- Первичный ключ (Primary Key) — уникальный идентификатор строки
- Внешний ключ (Foreign Key) — ссылка на первичный ключ другой таблицы

### 1.2. Типы данных
**Основные категории типов данных:**

| Категория     | Примеры                  | Описание                  |
|---------------|--------------------------|---------------------------|
| Строковые     | CHAR, VARCHAR, TEXT      | Текстовые данные          |
| Числовые      | INT, DECIMAL, FLOAT      | Числа                     |
| Дата/время    | DATE, TIME, TIMESTAMP    | Даты и время              |
| Логические    | BOOLEAN                  | Логические значения       |
| Бинарные      | BLOB, BINARY             | Бинарные данные           |
| JSON          | JSON, JSONB              | JSON документы            |

**Разница между CHAR и VARCHAR**
```sql
-- CHAR - фиксированная длина (дополняется пробелами)
CHAR(10) для ‘SQL’ → ‘SQL       ’ (7 пробелов)

-- VARCHAR - переменная длина (хранит только фактическое количество символов)
VARCHAR(10) для ‘SQL’ → ‘SQL’ (3 символа)

-- Рекомендации:
-- Использовать VARCHAR для текстовых полей
-- CHAR только для кодов фиксированной длины (ISO коды, аббревиатуры)
```

**Выбор типа данных:**
- INT для целых чисел (id, количество)
- DECIMAL(p,s) для денежных значений (p — точность, s — масштаб)
- VARCHAR(n) для текста, где n — максимальная длина
- DATE для дат без времени
- TIMESTAMP для даты и времени
- JSON/JSONB для неструктурированных данных (PostgreSQL: JSONB для бинарного хранения)

### 1.3. Ограничения (Constraints)
**Что такое PRIMARY KEY?**
Первичный ключ — столбец (или набор столбцов), уникально идентифицирующий каждую строку:
- Гарантирует уникальность значений
- Не допускает NULL значений
- Автоматически создает индекс (обычно кластеризованный)

```sql
CREATE TABLE users (
 id INT PRIMARY KEY,
 name VARCHAR(50)
);
```

**Для чего нужен FOREIGN KEY?**
Внешний ключ обеспечивает ссылочную целостность между таблицами:
- Связывает поле в дочерней таблице с PRIMARY KEY в родительской
- Предотвращает удаление связанных данных
- Обеспечивает целостность отношений

```sql
CREATE TABLE orders (
 order_id INT PRIMARY KEY,
 user_id INT,
 FOREIGN KEY (user_id) REFERENCES users(id)
 ON DELETE CASCADE
 ON UPDATE CASCADE
);
```

**Типы ограничений:**

| Ограничение | Описание | Пример |
|-------------|----------|--------|
| NOT NULL | Запрет пустых значений | name VARCHAR(50) NOT NULL |
| UNIQUE | Уникальность значений | email VARCHAR(100) UNIQUE |
| PRIMARY KEY | Первичный ключ (NOT NULL + UNIQUE) | id INT PRIMARY KEY |
| FOREIGN KEY | Внешний ключ | FOREIGN KEY (user_id) REFERENCES users(id) |
| CHECK | Проверка условия | age INT CHECK (age >= 18) |
| DEFAULT | Значение по умолчанию | created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP |

**Пример таблицы с ограничениями:**
```sql
CREATE TABLE employees (
 id INT PRIMARY KEY AUTO_INCREMENT,
 employee_code CHAR(10) UNIQUE NOT NULL,
 name VARCHAR(100) NOT NULL,
 email VARCHAR(100) UNIQUE NOT NULL,
 age INT CHECK (age >= 18 AND age <= 70),
 department_id INT,
 salary DECIMAL(10,2) DEFAULT 0,
 created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
 FOREIGN KEY (department_id) REFERENCES departments(id)
 ON DELETE SET NULL
);
```

### 1.4. CRUD операции
**CREATE — создание таблиц**
```sql
CREATE TABLE products (
 id INT PRIMARY KEY AUTO_INCREMENT,
 name VARCHAR(200) NOT NULL,
 price DECIMAL(10,2) CHECK (price >= 0),
 category VARCHAR(50),
 in_stock BOOLEAN DEFAULT TRUE,
 created_at DATETIME DEFAULT NOW()
);
```

**ALTER — изменение таблиц**
```sql
ALTER TABLE employees ADD COLUMN birth_date DATE;
ALTER TABLE employees MODIFY COLUMN name VARCHAR(150); -- MySQL
ALTER TABLE employees ALTER COLUMN name TYPE VARCHAR(150); -- PostgreSQL
```

**INSERT — добавление данных**
```sql
INSERT INTO products (name, price, category) VALUES
 ('Ноутбук', 1500.00, 'Электроника'),
 ('Стул', 200.00, 'Мебель');
```

**SELECT — выборка данных**
```sql
SELECT name, price FROM products WHERE price > 1000;
SELECT DISTINCT category FROM products;
```

**UPDATE — обновление данных**
```sql
UPDATE products SET price = price * 1.1 WHERE category = 'Электроника';
```

**DELETE — удаление данных**
```sql
DELETE FROM orders WHERE status = 'cancelled' AND created_at < '2024-01-01';
```

**Разница между DELETE, TRUNCATE и DROP**

| Операция | Скорость | Возврат данных | Автоинкремент | Где используется |
|-----------|----------|----------------|---------------|--------------------------------------|
| DELETE | Медленно | Да (ROLLBACK) | Сохраняется | Удаление строк с WHERE |
| TRUNCATE | Быстро | Нет | Сбрасывается | Удаление всех строк |
| DROP | Мгновенно| Нет | Удаляется | Удаление всей таблицы со структурой |

**Антипаттерны раздела 1**

❌ Таблица без первичного ключа
```sql
-- ПЛОХО
CREATE TABLE logs (message TEXT, created_at TIMESTAMP);
-- Проблемы: дубликаты, медленные JOIN, сложность обновления
```

✅ ХОРОШО
```sql
CREATE TABLE logs (
 id SERIAL PRIMARY KEY,
 message TEXT,
 created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

❌ Хранение нескольких значений в одном поле
```sql
-- ПЛОХО
product_ids VARCHAR(255) -- '1,5,23,42'
```

✅ ХОРОШО
```sql
CREATE TABLE order_items (
 order_id INT,
 product_id INT,
 FOREIGN KEY (order_id) REFERENCES orders(id),
 FOREIGN KEY (product_id) REFERENCES products(id)
);
```

❌ SELECT * в продакшене
```sql
-- ПЛОХО
SELECT * FROM users WHERE id = 1;
```

✅ ХОРОШО
```sql
SELECT id, name, email FROM users WHERE id = 1;
```

**Лучшие практики раздела 1**

- Всегда использовать первичные ключи (даже для логов)
- NOT NULL для обязательных полей
- UNIQUE для email, телефонов
- FOREIGN KEY для целостности
- Именование: users, created_at, is_active, user_id

**Различия MySQL и PostgreSQL**

| Концепция | MySQL | PostgreSQL |
|-----------|-------|------------|
| Автоинкремент | AUTO_INCREMENT | SERIAL или GENERATED ALWAYS AS IDENTITY |
| Логический тип | BOOLEAN = TINYINT(1) | BOOLEAN (нативный) |
| Ограничение длины | VARCHAR(255) обязательно | TEXT без ограничения |
| Каскадное удаление | ON DELETE CASCADE | ON DELETE CASCADE |

**Шпаргалка раздела 1**
```sql
CREATE TABLE users (
 id SERIAL PRIMARY KEY,
 email VARCHAR(150) UNIQUE NOT NULL,
 created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

INSERT INTO users (email) VALUES ('test@example.com');
SELECT * FROM users WHERE email = 'test@example.com';
```

## РАЗДЕЛ 2: ВЫБОРКА И ФИЛЬТРАЦИЯ ДАННЫХ

### 2.1. WHERE, ORDER BY, LIMIT/OFFSET

**Базовая структура запроса SELECT**
```sql
SELECT column1, column2, ... -- Что выбрать
FROM table_name -- Откуда выбрать
WHERE condition -- Условие фильтрации строк
ORDER BY column1 [ASC|DESC] -- Сортировка результатов
LIMIT number_of_rows; -- Ограничение количества строк
```

**Ключевое слово WHERE**
Фильтрует строки до их обработки и агрегации.
```sql
-- Простые условия
SELECT * FROM employees WHERE salary > 50000;
SELECT * FROM products WHERE price BETWEEN 20 AND 100;
SELECT * FROM users WHERE country IN ('Россия', 'Беларусь', 'Казахстан');

-- Комбинирование условий
SELECT * FROM orders
WHERE status = 'completed'
  AND total_amount > 1000
  AND order_date >= '2024-01-01';
```

**Сортировка с ORDER BY**
```sql
-- Сортировка по одному столбцу (по возрастанию ASC по умолчанию)
SELECT name, salary FROM employees ORDER BY salary;
-- Сортировка по убыванию
SELECT name, salary FROM employees ORDER BY salary DESC;
-- Сортировка по нескольким столбцам
SELECT first_name, last_name, department, salary
FROM employees
ORDER BY department ASC, salary DESC;
```

**Ограничение результатов LIMIT и OFFSET**
```sql
-- Получить первые 10 записей
SELECT * FROM products ORDER BY price DESC LIMIT 10;
-- Пагинация: пропустить 20 записей, взять следующие 10
SELECT * FROM products
ORDER BY created_at DESC
LIMIT 10 OFFSET 20; -- Страница 3 при размере страницы 10
-- Альтернативный синтаксис (MySQL)
SELECT * FROM products LIMIT 20, 10; -- OFFSET 20, LIMIT 10
```

**Порядок выполнения запроса для понимания:**
FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT/OFFSET

### 2.2. Операторы сравнения и логические операторы

**Операторы сравнения:**

| Оператор | Описание | Пример |
|----------|----------|--------|
| = | Равно | WHERE status = 'active' |
| <> или != | Не равно | WHERE status <> 'deleted' |
| < | Меньше | WHERE age < 18 |
| > | Больше | WHERE salary > 50000 |
| <= | Меньше или равно | WHERE quantity <= 10 |
| >= | Больше или равно | WHERE score >= 60 |
| BETWEEN | В диапазоне (включительно) | WHERE price BETWEEN 10 AND 100 |
| IN | В списке значений | WHERE country IN ('RU', 'BY', 'KZ') |
| LIKE | Соответствие шаблону | WHERE name LIKE 'Ив%' |
| IS NULL | Проверка на NULL | WHERE email IS NULL |

**Логические операторы:**
```sql
-- AND: оба условия истинны
SELECT * FROM users WHERE age >= 18 AND age <= 65;
-- OR: хотя бы одно условие истинно
SELECT * FROM products WHERE category = 'Электроника' OR price > 1000;
-- NOT: отрицание условия
SELECT * FROM orders WHERE NOT status = 'cancelled';
-- Комбинации с использованием скобок для контроля приоритета
SELECT * FROM employees
WHERE (department = 'IT' AND salary > 80000)
   OR (department = 'Sales' AND commission > 0.1);
```

**Приоритет операторов (от высшего к низшему):** Скобки () > Операторы сравнения > NOT > AND > OR

### 2.3. LIKE и регулярные выражения

**Оператор LIKE для поиска по шаблону:**
```sql
-- %: любое количество любых символов (0 или более)
SELECT * FROM users WHERE name LIKE 'Ив%'; -- Начинается на "Ив" (Иван, Игорь)
SELECT * FROM users WHERE email LIKE '%gmail.com'; -- Заканчивается на "gmail.com"
SELECT * FROM products WHERE name LIKE '%ноутбук%'; -- Содержит "ноутбук"

-- _: один любой символ
SELECT * FROM users WHERE phone LIKE '+7___%'; -- Российские номера (после +7 три любые цифры)
SELECT * FROM products WHERE sku LIKE 'ABC_2024'; -- ABC12024, ABC22024, и т.д.

-- Экранирование специальных символов
SELECT * FROM logs WHERE message LIKE '100\% complete'; -- Поиск "100% complete"
```

**Регулярные выражения (REGEXP):**
```sql
-- MySQL: REGEXP
SELECT * FROM users WHERE email REGEXP '^[a-z]+@gmail\.com$'; -- только gmail
SELECT * FROM products WHERE name REGEXP '^(Pro|Premium)'; -- начинается с Pro или Premium

-- PostgreSQL: ~ (регистрозависимый) и ~* (регистронезависимый)
SELECT * FROM users WHERE email ~ '^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$';
SELECT * FROM products WHERE name ~* 'iphone.*(14|15)'; -- iPhone 14 или 15, регистронезависимо
```

### 2.4. Работа с NULL значениями

**Что такое NULL?**
NULL означает "отсутствие значения" или "неизвестное значение"
NULL ≠ 0, NULL ≠ пустая строка '', NULL ≠ false
Любая операция с NULL возвращает NULL (NULL + 5 = NULL, NULL = NULL → NULL)

**Проверка на NULL:**
```sql
-- ❌ НЕПРАВИЛЬНО (всегда возвращает NULL, что интерпретируется как false)
SELECT * FROM employees WHERE phone = NULL; -- Не найдет ни одной строки
SELECT * FROM employees WHERE phone != NULL; -- Не найдет ни одной строки

-- ✅ ПРАВИЛЬНО: использовать IS NULL или IS NOT NULL
SELECT * FROM employees WHERE phone IS NULL;
SELECT * FROM employees WHERE phone IS NOT NULL;
```

**Функции для работы с NULL:**
```sql
-- COALESCE: возвращает первое не-NULL значение
SELECT
    name,
    COALESCE(phone, 'Не указан') AS phone_display,
    COALESCE(email, phone, 'Нет контактов') AS contact
FROM users;

-- IFNULL (MySQL) / ISNULL (SQL Server): аналог COALESCE для двух аргументов
SELECT name, IFNULL(phone, 'N/A') FROM users; -- MySQL

-- NULLIF: возвращает NULL если значения равны, иначе первое значение
SELECT NULLIF(sale_price, 0) AS effective_price FROM products; -- Заменяет 0 на NULL
```

**NULL в агрегатных функциях:**
```sql
-- Агрегатные функции игнорируют NULL (кроме COUNT(*))
SELECT
    AVG(commission) AS avg_commission, -- NULL не учитываются
    SUM(bonus) AS total_bonus, -- NULL игнорируются
    COUNT(commission) AS employees_with_commission, -- Только не-NULL
    COUNT(*) AS total_employees -- Все строки, включая NULL
FROM employees;
```

**Сортировка NULL значений:**
```sql
-- По умолчанию NULL считается наименьшим значением (идут первыми при ASC)
SELECT * FROM products ORDER BY price ASC; -- NULL будут первыми

-- Явное указание расположения NULL
SELECT * FROM employees
ORDER BY commission DESC NULLS LAST; -- NULL в конце
SELECT * FROM employees
ORDER BY commission ASC NULLS FIRST; -- NULL в начале (по умолчанию)
```

**Антипаттерны раздела 2**

❌ Использование функций в WHERE на индексированных полях
```sql
-- ПЛОХО: индекс не используется
SELECT * FROM users WHERE YEAR(created_at) = 2024;
SELECT * FROM users WHERE UPPER(name) = 'ИВАН';

-- ✅ ХОРОШО: переписать без функций
SELECT * FROM users WHERE created_at >= '2024-01-01' AND created_at < '2025-01-01';
SELECT * FROM users WHERE name = 'Иван';
```

❌ Неправильная проверка диапазонов дат
```sql
-- ПЛОХО: преобразование типов, индекс не используется
SELECT * FROM orders WHERE DATE(order_datetime) = '2024-01-15';

-- ✅ ХОРОШО: использовать диапазон
SELECT * FROM orders
WHERE order_datetime >= '2024-01-15 00:00:00'
  AND order_datetime < '2024-01-16 00:00:00';
```

❌ Использование OR вместо IN для многих значений
```sql
-- ПЛОХО: нечитаемо, может быть медленнее
SELECT * FROM products
WHERE category = 'Электроника'
   OR category = 'Бытовая техника'
   OR category = 'Мебель'
   OR category = 'Книги';

-- ✅ ХОРОШО: использовать IN
SELECT * FROM products
WHERE category IN ('Электроника', 'Бытовая техника', 'Мебель', 'Книги');
```

❌ SELECT * в больших таблицах
```sql
-- ПЛОХО: выборка всех столбцов, включая тяжелые (TEXT, BLOB)
SELECT * FROM products WHERE category = 'Электроника';

-- ✅ ХОРОШО: выбирать только необходимые столбцы
SELECT id, name, price, category
FROM products
WHERE category = 'Электроника';
```

**Лучшие практики раздела 2**

✅ Всегда использовать ORDER BY с LIMIT
```sql
-- Без ORDER BY результат непредсказуем
SELECT * FROM products LIMIT 10; -- ❌ Какие 10? Случайные!

-- С ORDER BY результат детерминирован
SELECT * FROM products ORDER BY created_at DESC LIMIT 10; -- ✅ Последние 10
```

✅ Правильное экранирование пользовательского ввода
Защита от SQL-инъекций — использовать параметризованные запросы в приложении.

✅ Использование EXPLAIN для анализа запросов
```sql
EXPLAIN SELECT * FROM users WHERE email LIKE '%@gmail.com';
```

✅ Создание индексов для часто фильтруемых полей
```sql
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
```

**Различия MySQL и PostgreSQL**

| Операция | MySQL | PostgreSQL |
|----------|-------|------------|
| Конкатенация строк | CONCAT('A', 'B') или 'A' || 'B' с режимом | 'A' || 'B' или CONCAT('A', 'B') |
| Регулярные выражения | REGEXP, RLIKE | ~, ~*, !~, !~* |
| LIMIT/OFFSET | LIMIT 10 OFFSET 20 или LIMIT 20, 10 | Только LIMIT 10 OFFSET 20 |
| Проверка на NULL | IS NULL | IS NULL |
| Сортировка NULL | NULL первые | NULL FIRST|LAST |
| ILIKE | Не поддерживается | ILIKE (регистронезависимый) |

**Шпаргалка раздела 2**

**Базовый запрос с фильтрацией:**
```sql
SELECT column1, column2
FROM table
WHERE condition
ORDER BY column1 DESC
LIMIT 10 OFFSET 20;
```

**Основные операторы WHERE:**
=, <>, <, >, <=, >= — сравнение
BETWEEN a AND b — диапазон (включительно)
IN (val1, val2, ...) — значение в списке
LIKE 'pattern%' — поиск по шаблону (% любое количество, _ один символ)
IS NULL, IS NOT NULL — проверка на NULL

**Логические операторы:**
AND — оба условия истинны
OR — хотя бы одно условие истинно
NOT — отрицание условия

**Приоритет:** () > NOT > AND > OR

**Функции работы с NULL:**
COALESCE(val1, val2, ...) — первое не-NULL значение
IFNULL(val, default) (MySQL) — заменяет NULL на default
NULLIF(val1, val2) — возвращает NULL если значения равны

**Важные замечания:**
- Всегда используйте ORDER BY с LIMIT
- Избегайте функций в WHERE на индексированных полях
- Для проверки на NULL используйте IS NULL, а не = NULL
- Выбирайте только нужные столбцы, а не SELECT *
- Используйте EXPLAIN для анализа медленных запросов